# XGBoost Built-in Algorithm

In [ ]:
import numpy as np
import pandas as pd

import boto3
import re

import sagemaker
from sagemaker import get_execution_role

## Upload Data to S3

In [ ]:
# Specify your bucket name
bucket_name = 'bike-rental-pellehardstedt'

training_folder = r'bikerental/training/'
validation_folder = r'bikerental/validation/'
test_folder = r'bikerental/test/'

s3_model_output_location = r's3://{0}/bikerental/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_folder)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_folder)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_folder)

In [ ]:
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [ ]:
write_to_s3('bike_train.csv', 
            bucket_name,
            training_folder + 'bike_train.csv')

write_to_s3('bike_validation.csv',
            bucket_name,
            validation_folder + 'bike_validation.csv')

write_to_s3('bike_test.csv',
            bucket_name,
            test_folder + 'bike_test.csv')

## Training Algorithm Docker Image

In [ ]:
sess = sagemaker.Session()

In [ ]:
role = get_execution_role()

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost",sess.boto_region_name,version="1.2-1")

print (f'Using XGBoost Container {container}')

## Build Model

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=s3_model_output_location,
    sagemaker_session=sess,
    base_job_name = 'xgboost-bikerental-v1')

In [ ]:
estimator.set_hyperparameters(max_depth=5,
                              objective="reg:squarederror",
                              eta=0.1,
                              num_round=150)

In [ ]:
estimator.hyperparameters()

### Specify Training Data Location and Optionally, Validation Data Location

In [ ]:
training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [ ]:
print(training_input_config.config)
print(validation_input_config.config)

### Train the model

In [ ]:
estimator.fit(data_channels)

## Deploy Model

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m4.xlarge',
                             endpoint_name = 'xgboost-bikerental-v1')

## Run Predictions

In [ ]:
from sagemaker.serializers import CSVSerializer

In [ ]:
predictor.serializer = CSVSerializer()

In [ ]:
predictor.predict([[3,0,1,2,28.7,33.335,79,12.998,2011,7,7,3]])